In [3]:
!pip install streamlit-option-menu

   ---------------------------------------- 0.0/823.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/823.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/823.4 kB ? eta -:--:--
   - -------------------------------------- 30.7/823.4 kB ? eta -:--:--
   - -------------------------------------- 30.7/823.4 kB ? eta -:--:--
   -- ------------------------------------ 61.4/823.4 kB 544.7 kB/s eta 0:00:02
   --- ----------------------------------- 81.9/823.4 kB 508.4 kB/s eta 0:00:02
   -------- ----------------------------- 194.6/823.4 kB 908.0 kB/s eta 0:00:01
   ---------- --------------------------- 225.3/823.4 kB 981.9 kB/s eta 0:00:01
   ---------- --------------------------- 225.3/823.4 kB 981.9 kB/s eta 0:00:01
   --------------------- ------------------ 450.6/823.4 kB 1.3 MB/s eta 0:00:01
   ------------------------ --------------- 501.8/823.4 kB 1.2 MB/s eta 0:00:01
   ------------------------------------- -- 778.2/823.4 kB 1.7 MB/s eta 0:00:01
   

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from streamlit_option_menu import option_menu
import mysql.connector as db
from query import *

st.set_page_config(page_title= "RedBus App", layout= "wide")
st.image("fc5b11fb-b135-4537-96ae-c3872b2164cb.jpeg")
st.title("Welcome to RedBus - My Seat My RedBus")

result = view_data()
col = [i[0] for i in curr.description]
df = pd.DataFrame(result, columns= col)

st.sidebar.image("redbus-app.webp", use_column_width= True, caption= "Redbus Web App")
st.sidebar.header("Please Filter Bus Route")
BusRouteName= st.sidebar.multiselect(
    "Select Bus Route Name", options= df["route_name"].unique()
    )

df_selection=df.query(
    "route_name==@BusRouteName" # & bus_type==@BusType"
    )

def Home():    
    col1,col2,col3,col4=st.columns(4, gap='large')
    with col1:
        top_bus_names = df_selection["bus_name"].value_counts().head(3).index.tolist()
        st.info('Top 3 Popular Bus Names:')
        for name in top_bus_names:
            st.write(name)
    with col2:
        top_bus_types= df_selection["bus_type"].value_counts().head(3).index.tolist()
        st.info('Top 3 Popular Bus Types:')
        for type in top_bus_types:
            st.write(type)
    with col3:
        Price_mean =df_selection["price"].mean()
        st.info("Average Price:")
        st.metric(label="Avg Price", value=f"{Price_mean:,.0f}")
    with col4:
        Star_rating =df_selection["star_rating"].mean()
        st.info("Average Star Rating:")
        st.metric(label="Avg Rating", value=f"{Star_rating:,.0f}")

    st.markdown("""---""")


def table():
    with st.expander("My Redbus Database"):
        shwdata=st.multiselect("Filter Dataset", df_selection.columns, default= ["bus_name", "bus_type", "departing_time", "duration","reaching_time","star_rating","price","seats_available"])
        st.dataframe(df_selection[shwdata], use_container_width = True)


with st.sidebar:
    menu = option_menu(
                menu_title = "Explore",
                options = ["Home","Route Links", "Dashboard", "Popular Cities"],
                styles = {
                    "nav-link-selected": {"background-color": "red"}
                }
    )
if menu=="Home":
    Home()
    table()
    
if menu=="Route Links":
    st.subheader("You selected Route Links")
    result1 = link()
    col = [i[0] for i in curr.description]
    data = pd.DataFrame(result1, columns= col)

    st.dataframe(data, column_config={"route_link": st.column_config.LinkColumn()})

if menu=="Dashboard":
    Price_by_bus_route= (df_selection.groupby(by=["route_name"]).count()[["price"]].sort_values(by="price"))
    fig_price=px.bar(
        Price_by_bus_route,x="price",y=Price_by_bus_route.index, orientation="h",
        title="Price by Bus Routes",
        template="plotly_white",
        color_discrete_sequence=px.colors.qualitative.Pastel,
    )
    fig_scatter = px.scatter(
        df_selection,
        x="price",
        y="bus_type",
        title="Price vs. Bus Type",
        template="plotly_white",
        opacity=0.7  # Adjust the opacity value as needed
    )
    left,right=st.columns(2)
    left.plotly_chart(fig_price,use_container_width=True)
    right.plotly_chart(fig_scatter,use_container_width=True)

if menu == "Popular Cities":
    popular_cities_data = get_popular_cities()
    popular_cities_df = pd.DataFrame(popular_cities_data, columns= [i[0] for i in curr.description])
    popular_cities_df = popular_cities_df.sort_values(by='route_name_count', ascending=False)

    # Create the line chart using Plotly Express
    fig = px.line(
        popular_cities_df,
        x='route_name',
        y='route_name_count',
        title='Popular Cities',
        markers=True  # Add markers for data points
    )

    # Customize the chart as needed (e.g., colors, labels, axis titles)
    fig.update_layout(
        xaxis_title='Route Name',
        yaxis_title='Route Name Count',
        xaxis_tickangle=-45  # Rotate x-axis labels for better readability
    )

    # Display the donut chart
    st.plotly_chart(fig,use_container_width=True)

    fig2= px.pie(
        df_selection,
        names="bus_type",
        values="price",
        title="Type of Buses wise Pricing",
        color_discrete_sequence=px.colors.sequential.Viridis,
        hole=0.4  # Adjust the hole size as needed
    )
    
    st.plotly_chart(fig2,use_container_width=True)

    
